In [1]:
import os
from openai import OpenAI
import pandas as pd

In [2]:
data_row=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/0504/gpt/train/sentence 4/train_data_before_vector.csv",index_col=0)
data_row

,sentence,predict
17414,"인천,김해,제주 공항에 국제운전면허 발급센터도 있습니다.",0
13248,국립중앙박물관 홈페이지에서 가능합니다.,0
2913,저희가 주민 번호 은행권에 계좌 번호 비밀 번호 등을 묻지 않는다고 말씀... 아...,2
13131,네 그러시군요.,0
817,피해자 :폐쇠한 것 같 아요 그거자체 를. 음 신한 은행은 신한 은행은 제가 기억...,2
...,...,...
1153,좀. 제가 지금까지 대출을 받은거 다 아시겠지만 받으면은 제외하거나 빼서 들어오지...,1
9952,물론입니다. 제보자님께 해가 가지않도록 비리신고제도를 운영하고 있고 보호 제도를 마...,0
2451,자녀분들의 나이가 어떻게 되시나요?,0
1430,발생한 비용에 대해서이자라든지 금액적인 부분에 대해서 본인께서 걱정하실 필요없구요...,2


In [7]:
data_row=data_row[:3000]

In [77]:
client = OpenAI(
    api_key = "key"
)


In [78]:
Y=pd.DataFrame(index=range(data_row.shape[0]),columns=['predict'])
gpt_data=pd.DataFrame(index=range(data_row.shape[0]),columns=['predict'])

In [61]:
Y

,predict
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
10239,NaN
10240,NaN
10241,NaN
10242,NaN


### 일반=0/대출사기형=1/수사기관형=2   gpt4 한번 질문하는데 2.2917076원

In [62]:
question="이 대화가 아주 정말 조금이라도 피싱이 의심이 되면 0을 출력하고 이 문장이 피싱이 정말 의심이 되지 않으면 1을 출력해, 0아님 1만 출력해"
A=" 이천 십 칠년 시 월 십육일 경남 마산 양덕동점에서 경남 은행과 국민은행 두 통장을 개설한 사실이 있습니까? 아니요 없으시고 두 통장은 불법 현장에서 발견되었기 때문에 본인 동의 없이 동결 처리된 상태입니다. 동결 처리는 어떤 뜻인지 알고 계신가요"+' '+question
A

' 이천 십 칠년 시 월 십육일 경남 마산 양덕동점에서 경남 은행과 국민은행 두 통장을 개설한 사실이 있습니까? 아니요 없으시고 두 통장은 불법 현장에서 발견되었기 때문에 본인 동의 없이 동결 처리된 상태입니다. 동결 처리는 어떤 뜻인지 알고 계신가요 이 대화가 아주 정말 조금이라도 피싱이 의심이 되면 0을 출력하고 이 문장이 피싱이 정말 의심이 되지 않으면 1을 출력해, 0아님 1만 출력해'

In [63]:
completion = client.chat.completions.create(
  model = "gpt-4-turbo-2024-04-09",
  messages = [
    {"role": "system", "content": A}
    
  ]
)
completion.choices[0].message.content.strip()

'0'

In [64]:
for i in range(data_row.shape[0]):#반복 횟수 설정? 바꿔야 할 수도
    completion = client.chat.completions.create(
    model = "gpt-4-turbo-2024-04-09",# model 바꿔야 할 수도 있음 우선 놔두고
    messages = [
        {"role": "system", "content": data_row.iloc[i,0]+' '+question}
    
        ]
    
    )
    Y.iloc[i,0]=completion.choices[0].message.content.strip()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [66]:
Y

,predict
0,1
1,1
2,0
3,0
4,1
...,...
1218,1
1219,1
1220,0
1221,0


In [ ]:
Y.to_csv("gpt predction data.csv")

In [3]:
Y=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/0504/gpt/train/use gpt/gpt predction data.csv",index_col=0)

In [4]:
Y

,predict
0,1
1,1
2,1
3,1
4,1
...,...
2995,1
2996,0
2997,1
2998,0


In [5]:
gpt_data# 0이면 보이스 피싱 1이면 피싱 아님

NameError: name 'gpt_data' is not defined

In [8]:
for i in range(data_row.shape[0]): #벡터화
    if Y.iloc[i,0]=='0':
        data_row.iloc[i,0]=data_row.iloc[i,0]+'본인'
    elif Y.iloc[i,0]=='1':
        data_row.iloc[i,0]=data_row.iloc[i,0]+'가능'

In [9]:
data_row.iloc[0,0]

'인천,김해,제주 공항에 국제운전면허 발급센터도 있습니다.가능'

In [10]:
data_row.to_csv("gpt train data before vector.csv")

In [11]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [12]:
A=model.encode(data_row.iloc[0,0])

In [13]:
list=[]

for i in range(A.shape[0]):
    list.append(i)

In [14]:
x_train=pd.DataFrame(index=range(0,data_row.shape[0]),columns=list)
y_train=pd.DataFrame(index=range(0,x_train.shape[0]),columns=['predict'])

In [15]:
for i in range(data_row.shape[0]):
    y_train.iloc[i,0]=data_row.iloc[i,1]

In [16]:
y_train

,predict
0,0
1,0
2,2
3,0
4,2
...,...
2995,1
2996,0
2997,0
2998,0


In [17]:
for i in range(data_row.shape[0]):
    vector=[]
    vector=model.encode(data_row.iloc[i,0])
    for j in range(x_train.shape[1]):
        x_train.iloc[i,j]=vector[j]

In [18]:
X=x_train

In [19]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.054102,-0.066255,-0.016948,-0.014147,0.002676,-0.004337,-0.02046,0.00221,0.072026,-0.016116,...,-0.045805,-0.039752,-0.01147,0.00796,-0.043131,-0.072515,0.016914,0.024208,0.015908,-0.016616
1,0.050208,-0.072811,-0.016652,-0.018788,-0.004397,-0.004018,-0.037195,0.011755,0.054287,-0.015238,...,-0.041961,-0.051621,-0.007345,0.010965,-0.039393,-0.049577,0.035705,0.022245,0.011584,-0.001973
2,0.074532,-0.003069,-0.029463,0.010639,0.003859,0.000853,-0.017487,0.010705,0.0831,-0.006545,...,-0.009486,0.003845,0.022685,-0.005338,-0.03702,-0.078075,0.011398,0.024196,0.018187,-0.031964
3,0.054348,-0.031571,-0.016646,-0.006379,0.035901,0.02096,-0.070056,0.024137,0.061972,-0.004382,...,-0.029347,-0.02656,0.00223,-0.01448,-0.051855,-0.072857,-0.005117,0.017494,0.027539,-0.019939
4,0.042024,-0.038339,-0.02986,0.026027,0.017407,-0.016309,-0.034739,0.026765,0.045727,0.009239,...,-0.062877,-0.001934,0.002913,0.014456,-0.014281,-0.009743,-0.00063,0.018181,-0.00241,-0.050316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.103314,-0.055173,-0.020592,0.040117,0.033047,0.013462,-0.041553,0.026483,0.07764,-0.022566,...,0.000349,-0.004895,-0.002905,0.013308,-0.044995,-0.075419,0.004471,0.01453,0.021715,-0.006937
2996,0.061798,-0.074197,-0.035188,-0.006774,0.025795,-0.003459,-0.001813,0.002595,0.045659,-0.009938,...,-0.037069,-0.038762,-0.015197,0.006406,-0.051039,-0.04718,0.00487,0.0252,0.006964,-0.029941
2997,0.03841,-0.094417,-0.026671,-0.021278,-0.008343,-0.00664,0.017228,-0.006007,0.056434,-0.007206,...,-0.040542,-0.039643,-0.026327,0.020404,-0.02964,-0.045425,0.022187,0.017653,-0.00258,-0.021919
2998,0.038844,-0.03809,-0.0281,0.011259,0.028264,-0.014,-0.063765,0.033624,0.039392,-0.020629,...,-0.034354,-0.012488,0.028551,-0.018211,-0.030303,-0.066341,0.023137,0.024972,0.007984,-0.018709


In [20]:
data_save=pd.concat([x_train,y_train],axis=1)
data_save

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,predict
0,0.054102,-0.066255,-0.016948,-0.014147,0.002676,-0.004337,-0.02046,0.00221,0.072026,-0.016116,...,-0.039752,-0.01147,0.00796,-0.043131,-0.072515,0.016914,0.024208,0.015908,-0.016616,0
1,0.050208,-0.072811,-0.016652,-0.018788,-0.004397,-0.004018,-0.037195,0.011755,0.054287,-0.015238,...,-0.051621,-0.007345,0.010965,-0.039393,-0.049577,0.035705,0.022245,0.011584,-0.001973,0
2,0.074532,-0.003069,-0.029463,0.010639,0.003859,0.000853,-0.017487,0.010705,0.0831,-0.006545,...,0.003845,0.022685,-0.005338,-0.03702,-0.078075,0.011398,0.024196,0.018187,-0.031964,2
3,0.054348,-0.031571,-0.016646,-0.006379,0.035901,0.02096,-0.070056,0.024137,0.061972,-0.004382,...,-0.02656,0.00223,-0.01448,-0.051855,-0.072857,-0.005117,0.017494,0.027539,-0.019939,0
4,0.042024,-0.038339,-0.02986,0.026027,0.017407,-0.016309,-0.034739,0.026765,0.045727,0.009239,...,-0.001934,0.002913,0.014456,-0.014281,-0.009743,-0.00063,0.018181,-0.00241,-0.050316,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.103314,-0.055173,-0.020592,0.040117,0.033047,0.013462,-0.041553,0.026483,0.07764,-0.022566,...,-0.004895,-0.002905,0.013308,-0.044995,-0.075419,0.004471,0.01453,0.021715,-0.006937,1
2996,0.061798,-0.074197,-0.035188,-0.006774,0.025795,-0.003459,-0.001813,0.002595,0.045659,-0.009938,...,-0.038762,-0.015197,0.006406,-0.051039,-0.04718,0.00487,0.0252,0.006964,-0.029941,0
2997,0.03841,-0.094417,-0.026671,-0.021278,-0.008343,-0.00664,0.017228,-0.006007,0.056434,-0.007206,...,-0.039643,-0.026327,0.020404,-0.02964,-0.045425,0.022187,0.017653,-0.00258,-0.021919,0
2998,0.038844,-0.03809,-0.0281,0.011259,0.028264,-0.014,-0.063765,0.033624,0.039392,-0.020629,...,-0.012488,0.028551,-0.018211,-0.030303,-0.066341,0.023137,0.024972,0.007984,-0.018709,0


In [21]:
data_save.to_csv("gpt train data.csv")